In [1]:
import os
import pathlib
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import folium

os.chdir("C:/Users/Potato/Sync/TED")
         
#open lists
with open('ph2.csv', newline='') as f:
    reader0 = csv.reader(f)
    phdata = list(reader0)
print(phdata[0])
with open('rem1.csv', newline='') as f:
    reader2 = csv.reader(f)
    exdata = list(reader2)
print(exdata[0])   

['Manila', '14.6', '120.9833', 'Manila', '23088000']
['ï»¿1', 'MC/FXD', 'CALUMPIT', 'BULACAN']


In [2]:
def ren(city):
    city=city.replace('ã±','ñ')
    city=city.replace('ã‘','ñ')
    city=city.lower()
    city=city.replace('city of ', '')
    city=city.replace(' city', '')
    city=city.split(' (')
    city=city[0]
    city=city.split('\n')
    city=city[0]
    city=city.replace('ã‘','ñ')
    city=city.replace('ã±','ñ')
    city=city.replace('ermita', 'manila')
    city=city.replace('binondo', 'manila')
    city=city.replace('tondo i/ii', 'manila')
    city=city.replace('malate', 'manila')
    city=city.replace('ermita', 'manila')
    city=city.replace('paco', 'manila')
    city=city.replace('port area', 'manila')
    city=city.replace('intramuros', 'manila')
    city=city.replace('quiapo', 'manila')
    city=city.replace('pandacan', 'manila')
    city=city.replace('sampaloc', 'manila')
    return city
for i in range(len(phdata)):
    phdata[i][0]=ren(phdata[i][0])
    phdata[i][3]=ren(phdata[i][3])
print(phdata[0])
rem=[]
for i in range(len(exdata)):
    exdata[i][2]=ren(exdata[i][2])
    exdata[i][3]=ren(exdata[i][3])
for k in exdata:
    if k[0]=='NO.':
        exdata.pop(exdata.index(k))
print(exdata[0])

['manila', '14.6', '120.9833', 'manila', '23088000']
['ï»¿1', 'MC/FXD', 'calumpit', 'bulacan']


In [3]:
mc=0
exdl=len(exdata)
notfound=[]
clean=[]
cleanarea=[]
for i in range(len(exdata)):
    found=0
    for j in range(len(phdata)):
        if exdata[i][2]==phdata[j][0]:
            found=2
    if found==0:
        notfound.append(exdata[i][2])
    if found!=0:
        clean.append(exdata[i])
        cleanarea.append(exdata[i][2])
        mc=mc+1
#print(exdl)
#print(mc)

In [4]:
from collections import Counter
Counter(notfound)

Counter({'sanchez-mira': 6,
         'cauayan': 17,
         'reina mercedes': 1,
         'cajidiocan': 3,
         'anini-y': 2,
         'libertad': 1,
         'mina': 1,
         'belison': 1,
         'bingawan': 1,
         'laguindingan': 3,
         'allen': 4,
         'calbiga': 8,
         'solsona': 1,
         'lebak': 2,
         'dapa': 4,
         'tubigon': 6,
         'burgos': 5,
         'general macarthur': 1,
         'marcos': 1,
         "brooke's point": 3,
         'marihatag': 1,
         'antipas': 2,
         'padada': 2,
         'labo': 7,
         'maigo': 2,
         'pantukan': 4,
         'sara': 4,
         'liloan': 9,
         'oroquieta': 9,
         'baloi': 1,
         'guihulngan': 6,
         'cuyo': 1,
         'bayambang': 8,
         'gen. mariano alvarez': 14,
         'tublay': 1,
         'san jose del': 38,
         'santa monica': 1,
         'sto. tomas': 10,
         'san benito': 2,
         'paoay': 2,
         'barcelona': 7,
   

In [5]:
cleancount=Counter(cleanarea)
cleandict=dict(cleancount)
cleankeys=list(cleandict.keys())
cleanvalues=list(cleandict.values())

In [6]:
visdata=[]
for i in range(len(cleankeys)):
    cur=[]
    cur.append(cleankeys[i])
    cur.append(cleanvalues[i])
    found=False
    for j in range(len(phdata)):
        if phdata[j][0]==cleankeys[i]:
            found=True
            cur=cur+phdata[j]
    visdata.append(cur)
#print(len(visdata))
#print(visdata[0])

In [7]:
vd=[]
for v in visdata:
    newv=[]
    newv.append(float(v[1]))
    newv.append(float(v[3]))
    newv.append(float(v[4]))
    newv.append(float(v[6]))
    newv.append(int(v[6])/int(v[1]))
    vd.append(newv)
print(vd[0])
df = pd.DataFrame(vd, columns = ['count', 'lat', 'long', 'pop', 'pop/ex'])
df.to_csv('done1.csv', encoding='utf-8')

[7.0, 14.9167, 120.7667, 108757.0, 15536.714285714286]


In [8]:
print(df.head())

counter=0
#generate map with circles representing population
map = folium.Map(location=[df['lat'].mean(), df['long'].mean()],
                    zoom_start=5)
for lat, long, pop, popex, count in zip(df['lat'], df['long'], df['pop'], df['pop/ex'], df['count']):
    if count<=5 and ((pop<1000000 and pop>50000) and (popex>20401 and popex<33495)): #approximately 60-70 percentile, less than 1m pop, 1/2 exchanges
        folium.CircleMarker(location=[lat, long],
                                radius=popex/6000, 
                                color='#FF0000',
                                fill_color='#FF0000',
                                fill_opacity=0.7).add_to(map)
        counter=counter+1
#     else:
#         folium.CircleMarker(location=[lat, long],
#                                 radius=1, 
#                                 color='#3186cc',
#                                 fill_color='#3186cc',
#                                 fill_opacity=0.7).add_to(map)
#save map
map.save('map.html')
print(counter)

   count      lat      long       pop        pop/ex
0    7.0  14.9167  120.7667  108757.0  15536.714286
1    6.0  14.8333  120.7333  129807.0  21634.500000
2   11.0  14.8333  120.8833   99730.0   9066.363636
3    3.0  14.7928  120.8789   76565.0  25521.666667
4    4.0  14.8667  120.9500   89075.0  22268.750000
66
